In [2]:
using JuMP, MosekTools, Gurobi
using SparseArrays, LinearAlgebra, Plots
using PowerModels, Ipopt

Academic license - for non-commercial use only


sample_pf (generic function with 2 methods)

In [71]:

include("../src/CVXRS_OPF.jl")

num_data=100
gamma_req=0.01


network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case89_pegase.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case162_ieee_dtc.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case200_tamu.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case240_pserc.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case300_ieee.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case500_tamu.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case588_sdet.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case1354_pegase.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case1888_rte.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case1951_rte.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case2000_tamu.m");

result_opf=run_opf(network_data, ACPPowerModel, with_optimizer(Ipopt.Optimizer,print_level=0))
println("Local search OPF objective solution: ",round(result_opf["objective"],digits=2))

## Initialize OPF
network_data=opf_initialization(network_data, 0.0)
[network_data["gen"][i]["ptc_factor"]=Int(network_data["bus"][string(gen["gen_bus"])]["bus_type"]==3) for (i,gen) in network_data["gen"]]
#[network_data["gen"][i]["ptc_factor"]=gen["gen_status"]*float((gen["pmax"]-gen["pmin"])>0) for (i,gen) in network_data["gen"]]

network_data=runpf(network_data)
test_runpf(network_data)
println("Initial OPF objective solution: ",round(network_data["cost"],digits=2))
violation_status, margin=check_violation(network_data)




Academic license - for non-commercial use only
[info | PowerModels]: removing 1 cost terms from generator 4: [3258.32, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 1: [658.654, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 12: [1765.53, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 2: [3479.49, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 6: [2770.7, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 11: [3127.13, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 5: [1676.25, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 7: [839.833, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 8: [3799.23, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 10: [2251.78, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 9: [2076.08, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 3: [4229.39, 0.0]


Local search OPF objective solution: 107285.67
Initial OPF objective solution: 107285.67


(Dict("line"=>[],"qg"=>[],"pg"=>[],"angle"=>[],"vmag"=>[]), 0.1)

In [72]:
## This is the power flow solving test
num_bus=length(network_data["bus"])
vm=zeros(Float64, num_bus); [vm[bus["idx"]]=bus["vm"] for (i,bus) in network_data["bus"]]
va=zeros(Float64, num_bus); [va[bus["idx"]]=bus["va"] for (i,bus) in network_data["bus"]]

network_data_test=deepcopy(network_data)
[gen["pg"]=gen["pg"]+gen["ptc_factor"]*network_data_test["delta"] for (i,gen) in network_data_test["gen"]]
result_test=run_pf(network_data_test, ACPPowerModel, with_optimizer(Ipopt.Optimizer,print_level=0))
vm_test=zeros(Float64, num_bus); [vm_test[bus["idx"]]=result_test["solution"]["bus"][i]["vm"] for (i,bus) in network_data_test["bus"]]
va_test=zeros(Float64, num_bus); [va_test[bus["idx"]]=result_test["solution"]["bus"][i]["va"] for (i,bus) in network_data_test["bus"]]

if max(maximum(abs.(vm-vm_test)),maximum(abs.(va-va_test)))>1e-5; @error "PowerFlow Failed" end;


In [58]:
maximum(abs.(vm-vm_test))

0.009053192296925916

In [59]:
maximum(abs.(va-va_test))

0.16676145445838111

In [60]:
sort(abs.(va-va_test))
mxval, mxindx = findmax(abs.(va-va_test))

(0.16676145445838111, 155)

In [53]:
bus_name=zeros(Float64, num_bus); [bus_name[bus["idx"]]=bus["bus_i"] for (i,bus) in network_data["bus"]]
bus_name[155]

2040.0

In [55]:

#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case89_pegase.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case179_goc.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case300_ieee.m");
#network_data = PowerModels.parse_file("../../../pglib-opf-master/pglib_opf_case1354_pegase.m");

num_line=length(network_data["branch"])
tap=zeros(Complex{Float64},num_line); [tap[branch["index"]]=branch["tap"]*exp(1im*pi/180*branch["shift"]) for (i,branch) in network_data["branch"]]
maximum(abs.(imag(tap)))

[info | PowerModels]: removing 1 cost terms from generator 32: [3101.67, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 29: [2727.85, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 1: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 54: [10398.7, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 2: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 41: [3965.77, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 65: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 51: [3512.67, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 53: [2999.2, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 27: [2839.15, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 42: [4122.33, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 33: [4972.18, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 28: [100.02, 0.0]
[info

0.003472631606021298

In [69]:

(Σ0,γ0)=network_data["uncertainty"]
network_data["uncertainty"]=(Σ0,gamma_req)
pl_sampled,data_gencost_bounded,violation_count_bounded,sample_violation_bounded=sample_pf(network_data,num_data,1)
count_bounded=violation_count_bounded
cost_worst_sampled=maximum(data_gencost_bounded)

(Σ0,γ0)=network_data["uncertainty"]
network_data["uncertainty"]=(Σ0,gamma_req/2)
pl_sampled,data_gencost_bounded,violation_count_normal,sample_violation_bounded=sample_pf(network_data,num_data,0)
count_normal=violation_count_normal

println("Violation probability (Bounded): ",count_bounded/num_data*100)
println("Violation probability (Normal): ",count_normal/num_data*100)


KeyError: KeyError: key "bus" not found

In [70]:
max_iter_SCRS=1

(Σ0,γ0)=network_data["uncertainty"]
network_data["uncertainty"]=(Σ0,gamma_req)

network_data, result_cvxr=scrs(network_data, max_iter_SCRS);

println("Robust OPF objective solution: ",round(network_data["cost"],digits=2))


(Σ0,γ0)=network_data["uncertainty"]
network_data["uncertainty"]=(Σ0,gamma_req)
pl_sampled,data_gencost_bounded,violation_count_bounded,sample_violation_bounded=sample_pf(network_data,num_data,1)
count_bounded=violation_count_bounded
cost_worst_sampled=maximum(data_gencost_bounded)

(Σ0,γ0)=network_data["uncertainty"]
network_data["uncertainty"]=(Σ0,gamma_req/2)
pl_sampled,data_gencost_bounded,violation_count_normal,sample_violation_bounded=sample_pf(network_data,num_data,0)
count_normal=violation_count_normal

println("Violation probability (Bounded): ", count_bounded/num_data*100)
println("Violation probability (Normal): ", count_normal/num_data*100)
println("Solver Time: ", sum(result_cvxr["solver_time"]))
println("Solver Time (1 iter): ", result_cvxr["solver_time"][2])




MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(2974) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(2975) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(2976) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(2977) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(2978) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(2979) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(2980) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(2981) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(2982) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(2983) of matrix 'A'.


Internal error: encountered unexpected error in runtime:
InterruptException()
jl_mutex_unlock at /Users/osx/buildbot/slave/package_osx64/build/src/./locks.h:138 [inlined]
jl_typeinf_end at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:2610
typeinf_ext at ./compiler/typeinfer.jl:577
typeinf_ext at ./compiler/typeinfer.jl:613
jfptr_typeinf_ext_1.clone_1 at /Applications/Julia-1.1.app/Contents/Resources/julia/lib/julia/sys.dylib (unknown line)
jl_apply_generic at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:2219 [inlined]
jl_apply at /Users/osx/buildbot/slave/package_osx64/build/src/./julia.h:1571 [inlined]
jl_type_infer at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:277
jl_compile_method_internal at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:1819
jl_fptr_trampoline at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:1863
add_bridged_constraint at /Users/dc/.julia/packages/MathOptInterface/A2UPd/src/Bridges/bridge_optimizer.jl:992
unknown fu

┌ Warning: vv out of range at bus 792: 1.097837 ≤ 1.097568 ≤ 1.101071
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:714
┌ Warning: g_vv out of range at bus 792: -1.099571 ≤ -1.09957 ≤ -1.099623
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:717
┌ Warning: Vm out of range at bus 5416/2: 1.051561 ≤ 1.052021 ≤ 1.051561
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:707
┌ Warning: vv out of range at bus 5416: 1.105759 ≤ 1.106748 ≤ 1.105407
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:714
┌ Warning: g_vv out of range at bus 5416: -1.104851 ≤ -1.10485 ≤ -1.104906
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:717
┌ Warning: Vm out of range at bus 6704/4: 1.006876 ≤ 1.00599 ≤ 1.009156
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:707
┌ Warning: Vm out of range at bus 1968/6: 1.029992 ≤ 1.026813 ≤ 1.026984
└ @ Main /Us

┌ Warning: g_vv out of range at bus 8229: -1.16449 ≤ -1.164486 ≤ -1.164538
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:717
┌ Warning: vv out of range at bus 7279: 1.13082 ≤ 1.130995 ≤ 1.130648
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:714
┌ Warning: Vm out of range at bus 8581/47: 1.053295 ≤ 1.054463 ≤ 1.053964
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:707
┌ Warning: g_vv out of range at bus 8581: -1.109823 ≤ -1.109822 ≤ -1.109877
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:717
┌ Warning: vv out of range at bus 8605: 1.033489 ≤ 1.033552 ≤ 1.033464
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:714
┌ Warning: g_vv out of range at bus 8605: 1.033482 ≤ 1.033552 ≤ 1.033263
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:717
┌ Warning: Vm out of range at bus 5848/49: 1.023403 ≤ 1.01935 ≤ 1.026441
└ @ Main

┌ Warning: g_vv out of range at bus 2267: 1.109179 ≤ 1.10918 ≤ 1.108841
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:717
┌ Warning: Vm out of range at bus 9192/88: 1.037738 ≤ 1.0279 ≤ 1.035623
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:707
┌ Warning: g_vv out of range at bus 9192: -1.071807 ≤ -1.071753 ≤ -1.07182
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:717
┌ Warning: Vm out of range at bus 2268/89: 1.003988 ≤ 0.995103 ≤ 1.004128
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:707
┌ Warning: g_vvcos out of range at line 24: -1.106991 ≤ -1.107423 ≤ -1.107548
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:747
┌ Warning: vvcos out of range at line 160: 1.136937 ≤ 1.125528 ≤ 1.130449
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:743
┌ Warning: g_vvcos out of range at line 160: -1.133492 ≤ -1.133923 ≤ -1.134

┌ Warning: vvcos out of range at line 138: 1.084319 ≤ 1.074397 ≤ 1.079872
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:743
┌ Warning: g_vvcos out of range at line 138: -1.081844 ≤ -1.082281 ≤ -1.082445
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:747
┌ Warning: vvcos out of range at line 153: 1.037374 ≤ 1.03621 ≤ 1.055941
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:743
┌ Warning: g_vvcos out of range at line 153: -1.048283 ≤ -1.048408 ≤ -1.048331
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:747
┌ Warning: Qg violation at bus 5097: -0.04 ≤ 0.34 ≤ 0.09
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:209
┌ Warning: Qg violation at bus 7960: -0.93 ≤ 3.8 ≤ 2.74
└ @ Main /Users/dc/Documents/Github/CVXRS_master/CVXRS_OPF/src/CVXRS_OPF.jl:209
┌ Warning: Qg violation at bus 9239: -0.93 ≤ 3.59 ≤ 2.74
└ @ Main /Users/dc/Documents/Github/CVXRS_mas

Iteration 1: 135845.8  139022.338   SLOW_PROGRESS    18.642   0.0163
Robust OPF objective solution: 135845.8


KeyError: KeyError: key "bus" not found